# 1. ChatMessageHistory的使用


## 场景1: 记忆存储

In [2]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory

history =  ChatMessageHistory()

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

print(history.messages)


[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]


## 场景2: 对接LLM

In [3]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain.memory import ChatMessageHistory

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)


history =  ChatMessageHistory()

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

history.add_user_message("帮我计算1 + 2 + 3 = ?")

response = chat_model.invoke(history.messages)

print(response.content)


按照数学运算顺序，从左到右依次计算：
先计算1 + 2 = 3，再计算3 + 3 = 6。

所以1 + 2 + 3的结果是6。 


# 2. ConversationBufferMemory

- 适用场景：对话轮次较少、依赖完整上下文的场景（如简单的聊天机器）
- 特点：
    - 完整存储对话历史
    - 简单 、 无裁剪 、 无压缩
    - 与 Chains/Models 无缝集成
    - 支持两种返回格式（通过 return_messages 参数控制输出格式）
        - return_messages=True 返回消息对象列表（ List[BaseMessage]
        - return_messages=False （默认） 返回拼接的 纯文本字符串



## 举例1: 以字符串的方式返回存储的信息

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(inputs={"input":"你好，我叫小明"},outputs = {"output":"很高兴认识你"})

memory.save_context(inputs={"input":"请帮我回到一下1 + 2 * 3 = ?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))
 

{'history': 'Human: 你好，我叫小明\nAI: 很高兴认识你\nHuman: 请帮我回到一下1 + 2 * 3 = ?\nAI: 7'}


## 举例2: 以消息列表的方式返回存储的消息

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(inputs={"input":"你好，我叫小明"},outputs = {"output":"很高兴认识你"})

memory.save_context(inputs={"input":"请帮我回到一下1 + 2 * 3 = ?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))

print('\n')

print(memory.chat_memory.messages)
 

{'history': [HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我回到一下1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}


[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我回到一下1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]
